<a href="https://colab.research.google.com/github/joe-mcnealPW/jiraConnector/blob/main/discovery_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!pip install nltk
!pip install pyLDAvis

In [ ]:
!pip install pyLDAvis==3.2.2


In [50]:
import numpy as np
import json
import glob
import pandas as pd

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
import nltk
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [7]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Helper Function Setup

In [8]:
def load_data(file_path):
  with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)
  return (data)

def write_data(file_path, data):
  with open(file_path, "w", encoding="utf-8") as file:
    json.dump(data, file, indent=4)

In [69]:
def unique_by_sh_msg(full_list):
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for item in full_list:
      short_message = item["short_message"]

      if short_message not in unique_list:
        unique_list.append(short_message)
    
    return unique_list

# Data Cleanup

In [72]:
data = load_data("/content/primary_data/discovery_logs.json")

#df = pd.DataFrame(data=data)
#short_messages = df.loc[:,"short_message"]

unique_data = unique_by_sh_msg(data)
print(len(data))
print(len(unique_data))
  

24073
1940


# Generate Bag of Words

In [74]:
def lemmatization(logs, allowed_postags=["NOUN","ADJ", "VERB", "ADV", "PROPN"]):
  nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  texts_out = []

  for log in logs:
    doc = nlp(log)
    new_text = []

    for token in doc:
      if token.pos_ in allowed_postags:
        new_text.append(token.lemma_)
      final = " ".join(new_text)
      texts_out.append(final)
  return (texts_out)

In [75]:
lemmatized_texts = lemmatization(unique_data)

In [76]:
print(lemmatized_texts[0:100])

['', 'host', 'host /partition_internal', 'host /partition_internal', 'host /partition_internal odiprod.trinet.com', 'host /partition_internal odiprod.trinet.com', 'host /partition_internal odiprod.trinet.com', 'host /partition_internal odiprod.trinet.com code]<a', 'host /partition_internal odiprod.trinet.com code]<a href="/nav_to.do?uri=%2fcmdb_ci.do%3fsys_id%3dcb8600d71353db840af6fa5ed144b02b', 'host /partition_internal odiprod.trinet.com code]<a href="/nav_to.do?uri=%2fcmdb_ci.do%3fsys_id%3dcb8600d71353db840af6fa5ed144b02b', 'host /partition_internal odiprod.trinet.com code]<a href="/nav_to.do?uri=%2fcmdb_ci.do%3fsys_id%3dcb8600d71353db840af6fa5ed144b02b target="_blank"><u>/partition_internal', 'host /partition_internal odiprod.trinet.com code]<a href="/nav_to.do?uri=%2fcmdb_ci.do%3fsys_id%3dcb8600d71353db840af6fa5ed144b02b target="_blank"><u>/partition_internal', 'host /partition_internal odiprod.trinet.com code]<a href="/nav_to.do?uri=%2fcmdb_ci.do%3fsys_id%3dcb8600d71353db840af6fa

In [14]:
def gen_words(texts):
  final = []
  for text in texts: 
    new = gensim.utils.simple_preprocess(text)
    final.append(new)
  
  return final

In [77]:
data_words = gen_words(lemmatized_texts)

print(data_words[1000:5000])

[['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip', 'mon'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip', 'mon', 'sep'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip', 'mon', 'sep'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip', 'mon', 'sep'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip', 'mon', 'sep'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip', 'mon', 'sep'], ['mon', 'sep', 'so', 'warn', 'pid', 'tid', 'ah', 'module', 'already', 'load', 'skip', 'mon', 'sep'], ['mon', 'sep', 'so', 'warn', 'pid', 't

In [89]:
print(len(data_words))

67842


In [97]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
  return (bigram[doc] for doc in texts)

def make_trigrams(texts):
  return (trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

In [100]:
phrases = bigram_phrases.export_phrases
print(b)

print(len(list(data_bigrams)))

<bound method Phrases.export_phrases of <gensim.models.phrases.Phrases object at 0x7f0dbe2af150>>
0


In [79]:
id2word = corpora.Dictionary(data_words)

In [80]:
corpus = []

for text in data_words:
  new = id2word.doc2bow(text)
  corpus.append(new)

print(corpus[0:100])

[[], [(0, 1)], [(0, 1)], [(0, 1)], [(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1

In [81]:
print(len(corpus))

67842


In [92]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto'
                                            )

In [93]:
pyLDAvis.enable_notebook()
print(pyLDAvis)
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

<module 'pyLDAvis' from '/usr/local/lib/python3.7/dist-packages/pyLDAvis/__init__.py'>


/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.261762  0.421128       1        1  45.039735
3      0.219255 -0.459204       2        1  22.237695
4      0.108607  0.465519       3        1   7.862454
0     -0.260746 -0.417043       4        1   7.356243
6     -0.268213  0.136740       5        1   5.885022
8      0.379580  0.275942       6        1   3.036369
7      0.488397 -0.105209       7        1   2.811548
2     -0.492688 -0.064877       8        1   2.488265
9      0.152827 -0.027575       9        1   2.040363
1     -0.065257 -0.225421      10        1   1.242304, topic_info=               Term          Freq         Total Category  logprob  loglift
93              sep  48484.000000  48484.000000  Default  30.0000  30.0000
117              dd  25287.000000  25287.000000  Default  29.0000  29.0000
421             tue  18048.000000  18048.000000  Default  28.0000  28.0000
106         pattern  63351.000000  63351.000000  Default  27.0000  27.0000
94               so  24211.000000  24211.000000  Default  26.0000  26.0000
..              ...           ...           ...      ...      ...      ...
358         husnmid     52.408564    729.582548  Topic10  -4.9878   1.7548
359   midserver_prd     52.408564    729.582548  Topic10  -4.9878   1.7548
505  identification      1.862472    958.259733  Topic10  -8.3250  -1.8550
507      context_id      1.437995    317.053191  Topic10  -8.5836  -1.0076
506  reconciliation      1.401680    730.791481  Topic10  -8.6092  -1.8683

[461 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
262       4  0.046212         aa
262       8  0.924247         aa
610       4  0.003917        aad
610       8  0.987069        aad
610      10  0.003917        aad
...     ...       ...        ...
692       9  0.839850   winrmapi
143       4  0.474962  wmiobject
73        4  0.474962        xml
685       4  0.013502      ysanv
685       5  0.972126      ysanv

[589 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 5, 1, 7, 9, 8, 3, 10, 2])